In [14]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

#vd.settings.default_backend = 'k3d'

## Initial Line Congruence test

### Visualization setup

In [ ]:
dir_path = os.getcwd()

data_path = dir_path+"/approximation/data/"

mv, mf = igl.read_triangle_mesh(os.path.join(data_path,"centers.obj" ))
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,"test_remeshed.obj" ))
# Get vertex normals for test mesh
tn = igl.per_vertex_normals(tv, tf)


In [ ]:

signs = np.sign(np.sum(tn * ([0,0,1]), axis=1))

# Fix normal directions
tn = tn * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Create dual mesh
mesh = Mesh()

mesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = mesh.inner_vertices()

# Dual topology including outer faces
dual = mesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]


# Create test mesh and center mesh 
mesh =  vd.Mesh((tv, tf), alpha = 0.8)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Create hexagonal mesh
hexmesh = vd.Mesh((h_pts, dual_faces), alpha = 0.5, c=[0.2, 0.4, 0.21])

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))
            
# Draw edges center mesh
center_edges = center.clone().wireframe().lw(1.5).flat().c('black').alpha(1)

# Draw edges Triangular mesh
edges = mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Draw axis lines
# Lines vd.Lines(ct- 5*nt, ct + 5*nt, c='red', lw=0.1)
clines = vd.Lines(ct[:5]- 5*nt[:5], ct[:5] + 5*nt[:5], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)
 

### Optimization Test

In [16]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")


# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])


# # Write data to file
for v  in tmesh.inner_vertices():
    file_pts.write(str(tmesh.vertices[v][0]) + " " + str(tmesh.vertices[v][1]) + " " + str(tmesh.vertices[v][2]) + "\n")
    ref.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")

file_pts.close()
ref.close()


# Optimization
# Init  X0 = [e_i | A| delta]
X = e_i.flatten() 

# Init constraints
linecong = LineCong()

linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)
 
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

for i in range(10):
    # Create file per iteration
    directions = open(os.path.join( out_path, "directions_"+str(k)+"_"+str(i)+".dat"), "w")

    # Compute J, r for each constraint
    linecong.compute(X,inner_vertices, dual)
    
    # Add constraints  
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    e_i = opt.X.reshape(-1,3)   

    # Write data to file
    for v  in inner_vertices:   
        directions.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
    directions.close()

# Print log
#opt.print_log()
e_i = opt.X.reshape(-1,3)   
    
# Visualize line congruence
#congruence = vd.Lines(tv[inner_vertices[:5]]- 5*e_i[inner_vertices[:5]], tv[inner_vertices[:5]] + 5*e_i[inner_vertices[:5]], c=[0.8,0.5,0.5], lw=0.1)


Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
 E 1: 78.03619781643371
 E 2: 19.968988507238908
 E 3: 5.2475803265827485
 E 4: 1.5957918188497011
 E 5: 0.6836355555692645
 E 6: 0.4555117063579754
 E 7: 0.39844183709265873
 E 8: 0.3841635158128383
 E 9: 0.3805911468100262
 E 10: 0.37969735152160855


# Torsal test

In [ ]:
tpos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
tdir = open(os.path.join( out_path, "torsal_dir_nor_"+str(k)+".dat"), "w")
bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)

tors, cos_tors = torsal_directions(tv, tf, e_i)
# Get even indices
tors1 = tors[::2]
# Get odd indices
tors2 = tors[1::2]

t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

inner_faces = t_mesh.inner_faces()

for ii in range(len(inner_faces)):
    i = inner_faces[ii]
    tpos.write(str(bar[i][0]) + " " + str(bar[i][1]) + " " + str(bar[i][2]) + "\n")
    tdir.write(str(t1[i][0]) + " " + str(t1[i][1]) + " " + str(t1[i][2]) + " " + str(t2[i][0]) + " " + str(t2[i][1]) + " " + str(t2[i][2]) + "\n")

tpos.close()
tdir.close()


badf = np.array([214,649,78,226,653])

print(f"e : \n{e_i[tf[badf]]} \n t1 :\n {t1[badf]} \n t2 :\n {t2[badf]} \n cos_tors :\n {cos_tors[badf]}")

#igl.write_triangle_mesh(os.path.join( out_path, "tri_"+str(k)+".obj"), tv, tf)

In [ ]:
# Initialize polyscope
ps.init()


# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True)
ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual_faces, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw circumcenter axis
# Register a point cloud
#v_cc = ps.register_point_cloud("Circumcenter ", tv, radius=0.01)

#print("ei :", e_i.shape)
#print("ct :", ct.shape)
# basic visualization
v_tmesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.0001, length=5, color=(0.8, 0.0, 0.2))


# View the point cloud and mesh we just registered in the 3D UI
ps.show()

# Optimization Hyperbolicity

## Test 1

In [3]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# tv test
tv = np.array([
    [0,0,1]  , # v0
    [1,0,0]  , # v1
    [0,1,0]  , # v2
    [0,0,-1] , # v3
    [-1,0,0] , # v4
    [0,-1,0] , # v5
    [-1,-1,0] # v6
    ])

# tf test
tf = np.array([
    [0,1,2],
    [0,2,3],
    [0,3,4],
    [0,4,5],
    [0,5,6]
    ])

# e_i test
e_i = np.array([
    [0,0,0.2],
    [0,0.4,0.4],
    [0.5,0.5,0.5],
    [0.5,0.5,-0.5],
    [0.5,-0.5,0.5],
    [0.5,-0.5,-0.5],
    [-0.5,0.0, 0.0]
    ])


# h pts test
h_pts = np.mean(tv[tf], axis=1) + np.cross(tv[tf[:,1]] - tv[tf[:,0]], tv[tf[:,2]] - tv[tf[:,0]]) / (2 * np.linalg.norm(np.cross(tv[tf[:,1]] - tv[tf[:,0]], tv[tf[:,2]] - tv[tf[:,0]]), axis=1))[:, None]


t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

dual = t_mesh.vertex_ring_faces_list()
inner_vertices = t_mesh.inner_vertices()

e_i = e_i / np.linalg.norm(e_i, axis=1)[:, None]

# Compute the directions at the barycenters
ec = np.sum( e_i[tf], axis = 1) / 3

# # Compute the edge vectors per each face
vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]


# # Compute the edge vectors per each face
vij = vj - vi
vik = vk - vi


eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]

# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# b = [eij, eik, ec]  c = [vij, vik, ec]

b = np.sum(eij*eikXec, axis=1)
c = np.sum(vij*vikXec, axis=1)

A = det1 + det2 

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
linecong = LineCong()
linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices, 0)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 4, 1)

# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X, "LM")

for i in range(20):

    # Add constraints
    opt.add_constraint(hyp, tf)
    opt.add_constraint(linecong, inner_vertices, dual)
    
    # Optimize
    opt.optimize()
    


e_i = opt.X[:3*len(tv)].reshape(-1,3)

print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# [vij, vik, ec] = det1 
vikXec = np.cross(vik, ec)
detA = np.sum(vij*vikXec, axis=1)

# [eij, eik, ec] = det2
eikXec = np.cross(eik, ec)
detC = np.sum(eij*eikXec, axis=1)
# b = [eij, eik, ec]  c = [vij, vik, ec]

A = det1 + det2 

Ac = opt.X[3*len(tv):3*len(tv)+len(tf)]

print(f"Disc f: {A**2- 4*detA*detC} \t delta: { opt.X[3*len(tv) + len(tf):]} \tenergy: {( A**2 - 4*detA*detC - 10)**2}")

print(f"Diff A {abs(Ac - A)}")

Mesh Data Structure: |V| = 7, |F| = 5, |E| = 11
 E 1: 1173.0156201595648
 E 2: 76.60973496775178
 E 3: 5.361873355657328
 E 4: 0.34839105135034265
 E 5: 0.012929187913963247
 E 6: 0.000487400405975572
 E 7: 1.9246236084818516e-05
 E 8: 7.678440581066279e-07
 E 9: 3.069776824299674e-08
 E 10: 1.227785163108042e-09
 E 11: 4.911049812398308e-11
 E 12: 1.964416503616275e-12
 E 13: 7.857678692060106e-14
 E 14: 3.143078609937859e-15
 E 15: 1.2572345863721904e-16
 E 16: 5.028949507709845e-18
 E 17: 2.0115815406496372e-19
 E 18: 8.046463309251824e-21
 E 19: 3.2184488498285867e-22
 E 20: 1.2873478723799676e-23
[1.46022315 1.26489184 1.44722012 1.61287943 1.24824862 1.34048478
 0.63238395]
Disc f: [10.62593097  7.66038732  5.54697011  2.91678575  0.71382303] 	 delta: [3.259744   2.76774047 2.35520065 1.70785999 0.84488048] 	energy: [ 0.39178958  5.47378748 19.82947517 50.17192412 86.23308279]
Diff A [3.88578059e-15 7.88258347e-15 5.55111512e-17 1.83186799e-15
 1.31006317e-14]


In [4]:
import polyscope as ps
import numpy as np

ps.init()
# ... setup, register a point cloud, etc ...
pts = ps.PointCloud("my points", np.random.rand(100,3))

#// Get a reference to your point cloud
ps_cloud = ps.get_point_cloud("my points")

# Add a random scalar quantity
N = ps_cloud.n_points()
vals = np.random.rand(N)
ps_cloud.add_scalar_quantity("test_vals", vals)

# Set the quantity as the point size
ps_cloud.set_point_radius_quantity("test_vals")
ps.show()

# Clear it out, go back to default constant size
ps_cloud.clear_point_radius_quantity()
ps.show()

# Set the quantity as the size using actual world-coordinate units,
# rather than auto-scaling the size

ps_cloud.set_point_radius_quantity("test_vals", autoscale=False)
ps.show()

In [11]:
import polyscope as ps
import polyscope.imgui as psim
import time


def run_opt():

    #for _ in range(it):
    # Add constraints
    opt.add_constraint(hyp, tf)
    opt.add_constraint(linecong, inner_vertices, dual)
    
    # Optimize
    opt.optimize()

    e_i = opt.X[:3*len(tv)].reshape(-1,3) + np.random.rand(3*len(tv)).reshape(-1,3)*0.5

    mesh = ps.get_surface_mesh("Mesh")
    mesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.01, length=0.2, color=(0.8, 0.0, 0.2))

        # Delay update
        #time.sleep(0.1)
        
        

def callback():

    # == Settings

    # Use settings like this to change the UI appearance.
    # Note that it is a push/pop pair, with the matching pop() below.
    psim.PushItemWidth(150)

    # == Show text in the UI
    psim.TextUnformatted("Optimization Gui")
    psim.TextUnformatted("An important value: {}".format(420))
    psim.Separator()


    # == Buttons
    if(psim.Button("Optimize?")):

        # run_opt(1)
        for _ in range(10):
            run_opt()
            ps.show()
        
    
        
        

    psim.PopItemWidth()

# Initialize polyscope
ps.init()
ps.set_always_redraw(True)
ps.set_invoke_user_callback_for_nested_show(False)
# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True, color=(0.8, 0.0, 0.8))

# basic visualization
v_tmesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.01, length=0.2, color=(0.8, 0.0, 0.2))

ps.set_user_callback(callback)

# View the point cloud and mesh we just registered in the 3D UI
ps.show()

ps.remove_all_structures()

 E 1081: 8.504906634414034e-31
 E 1082: 8.628166150854817e-31
 E 1083: 8.504906634414034e-31
 E 1084: 8.628166150854817e-31
 E 1085: 8.504906634414034e-31
 E 1086: 8.628166150854817e-31
 E 1087: 8.504906634414034e-31
 E 1088: 8.628166150854817e-31
 E 1089: 8.504906634414034e-31
 E 1090: 8.628166150854817e-31


## Test data

### Load Data

In [1]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong


# # Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")



# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = tmesh.inner_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326


In [2]:
# Compute the norms of ec
#nc = np.linalg.norm(ec, axis=1)
def compute_disc(tv, tf, e_i):

    # # Compute the edge vectors per each face
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]

    # # Compute the edge vectors per each face
    vij = vj - vi
    vik = vk - vi

    # Set up X 
    eij = e_i[tf[:,1]] - e_i[tf[:,0]]
    eik = e_i[tf[:,2]] - e_i[tf[:,0]]

    ec = np.sum( e_i[tf], axis = 1) / 3

    # A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
    # A = gamma11 +  gamma12
    eikXec = np.cross(eik, ec)
    vikXec = np.cross(vik, ec)

    det1 = np.sum(vij*eikXec, axis=1)
    det2 = np.sum(eij*vikXec, axis=1)

    # b = [eij, eik, ec]  c = [vij, vik, ec]

    gamma0 = np.sum(eij*eikXec, axis=1)
    gamma2 = np.sum(vij*vikXec, axis=1)

    A = det1 + det2 

    return A, A**2 - 4*gamma0*gamma2


A, disc = compute_disc(tv, tf, e_i)

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices, 1)

#np.random.seed(0)
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 1, 4)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(20):

    # Take time
    start = time.time()
    hyp.compute(X, tf)

    #print(hyp.r[hyp.r > 100 ])
    linecong.compute(X, inner_vertices, dual)

    opt.add_constraint(hyp)
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    X = opt.X

    

e_i = X[:3*len(tv)].reshape(-1,3)
A, disc = compute_disc(tv, tf, e_i)

delta = X[3*len(tv) + len(tf):]
print("Discriminant < 0:\n")
print(disc[disc<0])


 E 1: 3509.3508338953047
 E 2: 1105.2604758801167
 E 3: 348.5263463257316
 E 4: 110.16277046762518
 E 5: 35.01151901180181
 E 6: 11.304954331177395
 E 7: 3.824795420231912
 E 8: 1.4648090265775926
 E 9: 0.7206886365882563
 E 10: 0.48638707641087664
 E 11: 0.41279615380742135
 E 12: 0.38977020976510385
 E 13: 0.3826012492715146
 E 14: 0.38038169070950606
 E 15: 0.3796985220884871
 E 16: 0.37948959129246346
 E 17: 0.3794261822022558
 E 18: 0.37940711734714855
 E 19: 0.3794014477015665
 E 20: 0.3793997818491831
 E 21: 0.379399298546085
 E 22: 0.37939916011659597
 E 23: 0.3793991209684923
 E 24: 0.37939911003517796
 E 25: 0.3793991070838434
 E 26: 0.3793991063183611
 E 27: 0.37939910615095734
 E 28: 0.3793991062253341
 E 29: 0.3793991059990945
 E 30: 0.3793991062729919
 E 31: 0.3793991064358188
 E 32: 0.37939910608410415
 E 33: 0.3793991059722927
 E 34: 0.37939910596176424
 E 35: 0.37939910951176165
 E 36: 0.37939910707404445
 E 37: 0.3793991062931538
 E 38: 0.3793991061316875
 E 39: 0.379

### Visualization

In [13]:
e_i = opt.X[:3*len(tv)].reshape(-1,3)

bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)

tors, cos_tors1 = torsal_directions(tv, tf, e_i)

tt1 = tors[::2]
tt2 = tors[1::2]

print(cos_tors[np.where(cos_tors == -1)[0]])

# # # Initialize polyscope
ps.init()

# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True, color=(0.4, 0.7, 0.7))
#ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw baricenters
v_cc = ps.register_point_cloud("barycenters ", bar, radius=0.001)

# Add torsal directions
v_tmesh.add_vector_quantity("t1", t1, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.8, 0.0, 0.5))
v_tmesh.add_vector_quantity("-t1", -t1, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.8, 0.0, 0.5))
v_tmesh.add_vector_quantity("t2", t2, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.1, 0.1, 0.1))
v_tmesh.add_vector_quantity("-t2", -t2, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.1, 0.1, 0.1))
#v_tmesh.add_vector_quantity("t2", tt2, defined_on='faces', enabled=True, radius=0.005, length=0.01, color=(0.2, 0.0, 0.8))
#v_tmesh.add_vector_quantity("-t2", -tt2, defined_on='faces', enabled=True, radius=0.005, length=0.01, color=(0.2, 0.0, 0.8))

# View the point cloud and mesh we just registered in the 3D UI
ps.show()

[]


## Minimize Library test

In [ ]:
from scipy import optimize


def energy(x0, ei_dim, cicj, dual_faces):

    # energy \sum_{f \in F} \sum_{cj,ci \in E(f)} || e_f (cj - ci)  ||^2
    cf = dual_faces
    
    # Init energy value
    total_energy = 0

    # Get directions
    ei = x0.reshape(ei_dim, 3)

    # Loop over faces
    for f in range(len(cf)):
        # Define Jacobian
        diff = np.dot(cicj[f], ei[f])

        # Define residual
        total_energy += np.sum( diff**2, axis=0)
    
    diff2 = np.sum(ei * ei, axis=1) - 1

    total_energy += np.sum(diff2**2, axis=0)

    return total_energy

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create test mesh
test_mesh =  vd.Mesh((tv, tf), alpha = 0.8)

# Draw edges Triangular mesh
edges = test_mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual_faces = tmesh.dual_top()

# Dual topology without outer faces
#dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Compute cij
cij = []
# Loop over faces
for f in range(len(dual_faces)):
    # Get face
    face = dual_faces[f]

    # Get vertices
    v0 = h_pts[face]
    v1 = np.roll(h_pts[face], -1, axis=0)
 
    cicj = (v1 - v0)/np.linalg.norm(v1 - v0, axis=1)[:, None]
    # Define direction
    cij.append(cicj)

dim = len(inner_vertices)


opt = optimize.minimize(energy, e_i[inner_vertices].flatten(), args=(dim, cij, dual_faces), options={'disp': True})


In [ ]:
e_i = opt.x.reshape(-1,3)

minfile = open(os.path.join( out_path, "min_directions_"+str(k)+".dat"), "w")

for v  in range(len(tmesh.inner_vertices())):   
    minfile.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
minfile.close()



### Test Circumcircle

In [ ]:

# Get a list of random points
#p = np.random.rand(10,3)

p1 = np.random.rand(10,3)
p2 = np.random.rand(10,3)
p3 = np.random.rand(10,3)

c, rad, n = circle_3pts(p1,p2,p3)
print(f"centers: {c}\t rad: {rad}\t n: {n}")
circles = []
for ci in range(len(c)):
    print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array( np.vstack((p1, p2, p3))), r = 10, c = 'black')
#pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1,*circles)


## Sample Planarity Opt

In [ ]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

X = np.zeros((3*m.V+3*m.F), dtype=np.float64)

X[:3*m.V] = m.vertices.flatten()

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m, X)


# init optimizer
opt = Optimizer()

opt.initialize_optimizer(X)

for i in range(10):

    # Compute J, r for each constraint
    planarity.compute(m, X)
    
    # Add constraints  
    opt.add_constraint(planarity)
    opt.optimize("LM")
 

mesh2 = vd.Mesh((opt.X[:3*m.V].reshape(m.V,3),  fcs), alpha = 0.9, c='b')

print(opt.energy)

vd.show(mesh1, mesh2)



In [ ]:

v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

nv = np.array([[-0.08452795, -0.5184518 ,  0.1204968 ],
       [ 0.09325778, -0.5417252 , -0.14323919],
       [-0.39126414,  0.38199818,  0.21744937],
       [ 0.05301311,  0.4957555 , -0.02630325],
       [-0.04160952, -0.68063384,  0.11461049],
       [-0.64098644,  0.96492183,  0.3065665 ],
       [ 0.29607683,  0.33823755, -0.46029344],
       [-0.24634174, -0.29367885,  0.3622692 ],
       [ 0.5331171 , -1.4693466 , -0.43155265]]
       )


mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')
mesh2 = vd.Mesh((nv, fcs), alpha = 0.8, c='b')


vd.show(mesh1, mesh2)

In [ ]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

In [ ]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [ ]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

In [ ]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])
    vs.append(v)
    cs.append(c)
p